In [336]:
import pandas as pd
import datetime
import time
import os
import sqlite3
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv, find_dotenv
from langchain.schema import HumanMessage
from langchain.document_loaders import UnstructuredCSVLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEndpoint
import tiktoken  # Para contar tokens
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

In [345]:
load_dotenv(find_dotenv())
api_key = os.getenv("HUGGIN_FACE2")

# 2. Definir token e modelo
HUGGINGFACEHUB_API_TOKEN = api_key
MODEL_REPO_ID = "mistralai/Mistral-7B-Instruct-v0.2"
MAX_TOKENS = 8192

# 3. Template do prompt
template = """ 
    Você é um assistente especializado em futebol, capaz de responder perguntas de maneira precisa e informada.
    
    É muito importante que você responda da forma mais direta possível, você nao pode dar informações nas quais nao foram solicitados.
    Você precisa consultar os dados disponibilizados em {data}.
    Você não pode incluir códigos, a menos que seja pedido. 

    Lembrando que você tem todos os dados sobre o campeonato inglês (Premier League) e os dados junto dos significados das siglas que você tem à disposição estão dentro do arquivo {combined_info}.  
    
    Contexto:
    - Banco de dados: contém dados atualizados sobre jogadores, times, estatísticas e competições.
    - {combined_info}: inclui o glossário sobre siglas das colunas presentes nos dados.
    
    Arquivos com as estatísticas: {data}.
    Informações adicionais: {combined_info}.
    
    Pergunta do usuário: {question}
    
    Resposta:
    """

# 4. Função para carregar informações de arquivos
def load_info(file_paths):
    combined_content = ""
    for file in file_paths:
        with open(file, 'r', encoding='utf-8') as f:
            combined_content += f.read() + "\n"
    return combined_content

# 5. Carregar glossário e dados CSV
combined_info = load_info(file_paths=[
    r'C:\Users\Wellbe\DataspellProjects\dsProject\football_api\files\infos\glossary.txt'
])

loader = CSVLoader(
    file_path=r'C:\Users\Wellbe\DataspellProjects\dsProject\football_api\files\2024-12-20\data_info_pl.csv',
    encoding='latin1'
)
docs = loader.load()

# 6. Dividir os dados em partes menores
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

# 7. Calcular tokens e garantir que fiquem dentro do limite
tokenizer = tiktoken.get_encoding("gpt2")
valid_chunks = []
for doc in split_docs:
    num_tokens = len(tokenizer.encode(doc.page_content))
    if num_tokens + 256 <= MAX_TOKENS:  # Garante que entrada + saída não ultrapassem 8192
        valid_chunks.append(doc)

# 8. Criar embeddings e indexar os documentos
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(valid_chunks, embedding_model)

# 9. Função para recuperar documentos relevantes
def retrieve_relevant_docs(question, top_k=3):
    return vector_store.similarity_search(question, top_k)

# 10. Configuração do modelo com HuggingFaceEndpoint
llm = HuggingFaceEndpoint(
    endpoint_url=f"https://api-inference.huggingface.co/models/{MODEL_REPO_ID}",
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
    temperature=0.2,
    max_new_tokens=512
)

# 11. Função para gerar respostas com RAG
def generate_answer(question):
    # Recuperar documentos relevantes
    relevant_docs = retrieve_relevant_docs(question)
    context = "\n".join([doc.page_content for doc in relevant_docs])

    # Criar o prompt com as informações recuperadas
    prompt = PromptTemplate(
        template=template,
        input_variables=['question', 'data', 'combined_info']
    )
    prompt_with_context = prompt.format(
        question=question,
        data=context,
        combined_info=combined_info
    )

    # Gerar a resposta usando o LLM
    response = llm.invoke(prompt_with_context)
    return response



In [346]:
# 12. Testar o pipeline com uma pergunta
question = "Me faça uma lista dos jogadores do chelsea disponiveis na base de dados?"
answer = generate_answer(question)
print(answer)


    Based on the provided data, the following Chelsea players are available in the database:

    1. Moisés Caicedo (ECU, MF/DF, Age: 23)
    2. Tosin Adarabioyo (ENG, DF, Age: 27)
    3. João Félix (POR, FW/MF, Age: 25)

    These players have all appeared in the Premier League matches for Chelsea and their statistics are available in the database.


In [332]:
# Pergunta sobre estatísticas de futebol
#question = "Me faça uma lista dos times disponiveis na base de dados?"
#question = "Me faça uma lista dos jogadores do chelsea disponiveis na base de dados?"
question = "Qual jogador do chelsea com mais gols?"


